Mean Absolute Error (MAE)

In [ ]:
!pip install statsmodels tensorflow

import pandas as pd
from sklearn.metrics import mean_absolute_error
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/electric-car-sales-share.csv'
df = pd.read_csv(file_path)
df.rename(columns={'Entity': 'Country', 'Share of new cars that are electric': 'Sales'}, inplace=True)

countries = ['United States', 'Japan', 'India', 'China', 'United Kingdom', 'Germany','France']
df_filtered = df[df['Country'].isin(countries)]
average_sales_share = df_filtered.groupby('Country')['Sales'].mean()
print("Average Electric Car Sales Share:")
print(average_sales_share)



df['Year'] = pd.to_datetime(df['Year'], format='%Y')  
df.set_index('Year', inplace=True)
df = df[['Sales']]  

train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)

arima_model = ARIMA(train_data, order=(5, 1, 0))
arima_result = arima_model.fit()
arima_forecast = arima_result.forecast(steps=len(test_data))
arima_mae = mean_absolute_error(test_data, arima_forecast)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

X_train = []
y_train = []
time_step = 5

for i in range(time_step, len(scaled_data) - len(test_data)):
    X_train.append(scaled_data[i - time_step:i, 0])
    y_train.append(scaled_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_absolute_error')

lstm_model.fit(X_train, y_train, epochs=10, batch_size=32)

X_test = []
for i in range(len(scaled_data) - len(test_data), len(scaled_data)):
    X_test.append(scaled_data[i - time_step:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

lstm_forecast = lstm_model.predict(X_test)
lstm_forecast = scaler.inverse_transform(lstm_forecast) 
lstm_mae = mean_absolute_error(test_data, lstm_forecast)

results_df = pd.DataFrame({
    'Model': ['Proposed Hybrid Model (LSTM)', 'ARIMA'],
    'Mean Absolute Error (MAE)': [lstm_mae, arima_mae]
})

print("Results:")
print(results_df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Average Electric Car Sales Share:
Country
China             7.422286
France            5.977286
Germany           7.422636
India             0.281157
Japan             1.034500
United Kingdom    6.303929
United States     2.382286
Name: Sales, dtype: float64


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsm

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0633
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0747
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0712
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0672
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0595
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0623
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0650
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0740
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0724
Epoch 10/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0676
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step
Results:
                          Model  Mean Absolute Error (MAE)
0  Proposed Hybrid Model (LSTM)                   5.473879
1                         ARIMA                   5.588301


Mean Squared Error (MSE)

In [ ]:
!pip install statsmodels tensorflow

import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/electric-car-sales-share.csv' 
df = pd.read_csv(file_path)
df.rename(columns={'Entity': 'Country', 'Share of new cars that are electric': 'Sales'}, inplace=True)

countries = ['United States', 'Japan', 'India', 'China', 'United Kingdom', 'Germany', 'France']

results = {}

for country in countries:
    df_country = df[df['Country'] == country].copy()
    df_country['Year'] = pd.to_datetime(df_country['Year'], format='%Y')
    df_country.set_index('Year', inplace=True)
    df_country.index.freq = 'YS'  

    train_data, test_data = train_test_split(df_country[['Sales']], test_size=0.2, shuffle=False)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df_country[['Sales']])

    X_train = []
    y_train = []
    time_step = 3 

    for i in range(time_step, len(scaled_data) - len(test_data)):
        X_train.append(scaled_data[i - time_step:i, 0])
        y_train.append(scaled_data[i, 0])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    lstm_model = Sequential()
    lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    lstm_model.add(LSTM(units=50))
    lstm_model.add(Dense(1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error') 

    lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    X_test = []
    for i in range(len(scaled_data) - len(test_data), len(scaled_data)):
        X_test.append(scaled_data[i - time_step:i, 0])

    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    lstm_forecast = lstm_model.predict(X_test)
    lstm_forecast = scaler.inverse_transform(lstm_forecast)
    lstm_mse = mean_squared_error(test_data, lstm_forecast)

    results[country] = {'LSTM_MSE': lstm_mse}

print("Country\tLSTM MSE")
for country, metrics in results.items():
    print(f"{country}\t{metrics['LSTM_MSE']:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step
Country	LSTM MSE
United States	26.4404
Japan	3.7647
India	1.7106
China	470.3197
United Kingdom	109.2277
Germany	241.3088
France	134.3687
